<a href="https://colab.research.google.com/github/IraStreltsova/MissionSquare_Surveys/blob/main/PPT_CONTACT_CENTER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd

In [42]:
# Constants
GENERAL_DATA_FILE = 'MSQ Survey General July 2023.xlsx'
VRS_DATA_FILE = 'VRS Survey Results July 2023.xlsx'
MO_DATA_FILE = 'MO Survey Results July 2023.xlsx'
VALID_SAT_SCORES = [1, 2, 3, 4]
VALID_CALL_SCORES = {
    'Q1': [1, 2, 3, 4, 5, 6, 7, 9],
    'Q7': [1, 2],
}
RES_SAT_COLUMNS = ['MTD', 'TOP 1 BOX %', 'TOP 2 BOX %', 'n']
RES_CALL_COLUMNS = ['MTD', 'n']
general_sat_columns= {
  'Q6 - Overall': 'Survey.Mission Q6 Results',
  'Q2 - Informative': 'Survey.Mission Q2 Results',
  'Q3 - Helpful': 'Survey.Mission Q3 Results',
  'Q4 - Knowledgeable': 'Survey.Mission Q4 Results',
  'Q5 - Professional': 'Survey.Mission Q5 Results',
}
vrs_sat_columns= {
  'Q6 - Overall': 'question6Value',
  'Q2 - Informative': 'question2Value',
  'Q3 - Helpful': 'question3Value',
  'Q4 - Knowledgeable': 'question4Value',
  'Q5 - Professional': 'question5Value',
}
mo_sat_columns= {
  'Q2 - Informative': 'question2Value',
  'Q3 - Helpful': 'question3Value',
  'Q6 - Overall': 'question6Value',
  'Q5 - Professional': 'question5Value',
  'Q4 - Knowledgeable': 'question4Value',
}
general_call_columns= {
  'Q1': 'Survey.Mission Q1 Results',
  'Q7': 'Survey.Mission Q7 Results',
}
call_columns= {
  'Q1': 'question1Value',
  'Q7': 'question7Value',
}
RESULTS_FILE = 'PPT_CONTACT_CENTER_RESULTS.xlsx'

In [43]:
def read_and_clean_data(file_path):
  """Read and clean data from the Excel file."""
  df = pd.read_excel(file_path)
  columns_to_keep = [col for col in df.columns if 'Q' in col or col == 'SKILL' or 'question' in col]
  df = df[columns_to_keep]
  df = df.fillna(0)
  #df.iloc[:, 1:] = df.iloc[:, 1:].astype(int)
  return df

In [44]:
def calculate_sat_metrics(df, column_name, filter_condition=None):
  """Calculate SAT metrics: MTD, TOP 1 BOX %, TOP 2 BOX %, and n."""
  output_df = pd.DataFrame(columns=RES_SAT_COLUMNS)

  # Filter rows where the column contains values 1, 2, 3, 4
  valid_data = df[df[column_name].isin(VALID_SAT_SCORES)]

  if filter_condition:
        valid_data = valid_data[valid_data['SKILL'] == filter_condition]

  # AVERAGE
  avg_score = valid_data[column_name].mean()

  # TOP 1 AND TOP 2 BOX
  # Group column 'Survey.Mission Helpful Results' and calculate percentages
  summary = valid_data[column_name].value_counts(normalize=True)

  # Calculate percentages for 'TOP 1 BOX %' and 'TOP 2 BOX %'
  top_1_box = summary.get(4, 0)
  top_2_box = top_1_box + summary.get(3, 0)

  # NUMBER OF ROWS
  n = len(valid_data)

  # APPEND DATA to the output DataFrame
  output_df = output_df.append({
    'MTD': avg_score,
    'TOP 1 BOX %': top_1_box,
    'TOP 2 BOX %': top_2_box,
    'n': n
  }, ignore_index=True)

  return output_df

In [45]:
def calculate_call_metrics(df, column_name, valid_scores, territory_condition=None):
  """Calculate SAT metrics: MTD and n."""
  output_df = pd.DataFrame(columns=RES_CALL_COLUMNS)

  # Filter rows where the column contains values
  valid_data = df[df[column_name].isin(valid_scores)]

  if territory_condition:
        valid_data = valid_data[valid_data['SKILL'] == territory_condition]

  # Group column 'Survey.Mission Helpful Results' and calculate percentages
  summary = valid_data[column_name].value_counts(normalize=True)
  call_dict = dict(sorted(summary.items()))

  # NUMBER OF ROWS
  n = len(valid_data)

  call_dict.update({'n': n})

  return call_dict

In [46]:
def process_sat_metrics(df, columns, territory_condition=None):
    result = {}
    for row_name, column_name in columns.items():
        metrics = calculate_sat_metrics(df, column_name, territory_condition)
        result[row_name] = pd.DataFrame(metrics, index=[0])

    df_list = []
    for index_name, dataframe in result.items():
        dataframe.index = pd.MultiIndex.from_product([[index_name], dataframe.index])
        df_list.append(dataframe)

    merged = pd.concat(df_list)
    merged = merged.droplevel(level=1, axis=0)
    return merged

In [47]:
def process_call_metrics(df, columns, valid_scores, filter_condition=None):
    result = {}
    for row_name, column_name in columns.items():
        metrics = calculate_call_metrics(df, column_name, valid_scores, filter_condition)
        result[row_name] = pd.DataFrame(metrics, index=[0])

    df_list = []
    for index_name, dataframe in result.items():
        dataframe.index = pd.MultiIndex.from_product([[index_name], dataframe.index])
        df_list.append(dataframe)

    merged = pd.concat(df_list)
    merged = merged.droplevel(level=1, axis=0)
    merged = merged.T
    return merged

In [48]:
def export_to_excel(dataframes, file_path):
    with pd.ExcelWriter(file_path) as writer:
        for name, df in dataframes.items():
            df.to_excel(writer, sheet_name=name, index=True)

In [49]:
def main():

  # GENERAL AND DC
  # Read the Excel file into a DataFrame
  df_general = read_and_clean_data(GENERAL_DATA_FILE)

  # Process general metrics
  general = process_sat_metrics(df_general, general_sat_columns)

  # Process DC metrics
  dc = process_sat_metrics(df_general, general_sat_columns, 'Mission_EE_DC_A')

  Q1_gen = dict(filter(lambda item: item[0] in 'Q1', general_call_columns.items()))
  Q7_gen = dict(filter(lambda item: item[0] in 'Q7', general_call_columns.items()))

  general_call = process_call_metrics(df_general, Q1_gen, VALID_CALL_SCORES['Q1'])
  dc_call = process_call_metrics(df_general, Q1_gen, VALID_CALL_SCORES['Q1'], 'Mission_EE_DC_A')

  general_Q7 = process_call_metrics(df_general, Q7_gen, VALID_CALL_SCORES['Q7'])
  dc_Q7 = process_call_metrics(df_general, Q7_gen, VALID_CALL_SCORES['Q7'], 'Mission_EE_DC_A')



  # VRS
  df_vrs = read_and_clean_data(VRS_DATA_FILE)

  # Process general metrics
  vrs = process_sat_metrics(df_vrs, vrs_sat_columns)

  Q1_vrs = dict(filter(lambda item: item[0] in 'Q1', call_columns.items()))
  Q7_vrs = dict(filter(lambda item: item[0] in 'Q7', call_columns.items()))

  vrs_call = process_call_metrics(df_vrs, Q1_vrs, VALID_CALL_SCORES['Q1'])
  vrs_Q7 = process_call_metrics(df_vrs, Q7_vrs, VALID_CALL_SCORES['Q7'])




  # Missouri
  df_mo = read_and_clean_data(MO_DATA_FILE)

  # Process general metrics
  mo = process_sat_metrics(df_mo, mo_sat_columns)

  Q1_mo = dict(filter(lambda item: item[0] in 'Q1', call_columns.items()))

  mo_call = process_call_metrics(df_mo, Q1_mo, VALID_CALL_SCORES['Q1'])


  df_to_export = {
    'GENERAL': general,
    'GENERAL_CALL': general_call,
    'GENERAL_Q7': general_Q7,
    'DC': dc,
    'DC_CALL': dc_call,
    'DC_Q7': dc_Q7,
    'VRS': vrs,
    'VRS_CALL': vrs_call,
    'VRS_Q7': vrs_Q7,
    'Missouri': mo,
    'Missouri_CALL': mo_call,
  }


  export_to_excel(df_to_export, RESULTS_FILE)


In [50]:
if __name__ == "__main__":
  main()

<ipython-input-44-dfde6ee82bd6>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({
<ipython-input-44-dfde6ee82bd6>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({
<ipython-input-44-dfde6ee82bd6>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({
<ipython-input-44-dfde6ee82bd6>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({
<ipython-input-44-dfde6ee82bd6>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df